### Erik's Fortran tracking code

In [ ]:
program main
!   
   use netcdf
!
   character(len=128)::filedir,filein,fileout
!
   integer::nlon,nlat,nt,nlons,nlats,status,ncid,varid,i,j,k,l,ii,jj,kk,nlow
   real,allocatable::lon(:),lat(:),varin(:,:,:)
   real,allocatable::varout(:,:,:,:)
   real, allocatable::dist(:,:,:),w(:,:,:,:),coarse(:,:),smooth(:,:,:)
   real,allocatable::smoothext(:,:),freq(:,:,:)
   real::pi,phi1,phi21,phi22,dlon,dlam,sigma(2),temp,tempmax
   real::compare(3,3),lowinfo(4,1000)
! 
   nlon = 512; nlat = 128; nt = 28
   nlons = 50; nlats = 29
!
! e-folding range for Gaussian smoothing
   sigma = (/150.0,2000.0/)
!
   pi = acos(-1.0)
!
   allocate(lon(nlon),lat(nlat))
   allocate(coarse(nlon+2*nlons,nlat),smooth(nlon+2,nlat-2*nlats,3))
   allocate(smoothext(nlon+2*nlons,nlat))
   allocate(dist(nlons+1+nlons,nlats+1+nlats,nlat-2*nlats))
   allocate(w(nlons+1+nlons,nlats+1+nlats,nlat-2*nlats,5))
   allocate(freq(nlon+2*nlons,nlat,3))
   allocate(varin(nlon,nlat,nt),varout(nlon,nlat,nt,3))
!
   filedir = '/shared/scratch/swenson/feature_detection/'
   filein  = trim(filedir)//'erai.week.2015.nc'
   fileout = trim(filedir)//'feature.erai.week.2015.nc'
!
! read in data
   status = nf90_open(trim(filein),NF90_NOWRITE,ncid)
   status = nf_inq_varid(ncid,'lon',varid)
   status = nf90_get_var(ncid,varid,lon)
   status = nf_inq_varid(ncid,'lat',varid)
   status = nf90_get_var(ncid,varid,lat)
   status = nf_inq_varid(ncid,'vor850',varid)
   status = nf90_get_var(ncid,varid,varin)
   status = nf90_close(ncid)
!
! compute arc length distance over sub-grid as function of latitude for central grid point
!
! longitude width in radians
   dlon = (360.0/real(nlon))*pi/180.0
   dist(:,:,:) = 0.0
   do j=(nlats+1),(nlat-nlats)
      phi1 = lat(j)*pi/180.0
      do jj=1,nlats
         phi21 = lat(j-jj)*pi/180.0
         phi22 = lat(j+jj)*pi/180.0
         do ii=1,nlons
            dlam = real(ii)*dlon
            temp = 6370.0*acos(sin(phi1)*sin(phi21)+cos(phi1)*cos(phi21)*cos(dlam))
            dist(nlons+1-ii,nlats+1-jj,j-nlats) = temp
            dist(nlons+1+ii,nlats+1-jj,j-nlats) = temp
            temp = 6370.0*acos(sin(phi1)*sin(phi22)+cos(phi1)*cos(phi22)*cos(dlam))
            dist(nlons+1-ii,nlats+1+jj,j-nlats) = temp
            dist(nlons+1+ii,nlats+1+jj,j-nlats) = temp
         end do
! at same longitude
         dlam = 0.0
         temp = 6370.0*acos(sin(phi1)*sin(phi21)+cos(phi1)*cos(phi21))
         dist(nlons+1,nlats+1-jj,j-nlats) = temp
         temp = 6370.0*acos(sin(phi1)*sin(phi22)+cos(phi1)*cos(phi22))
         dist(nlons+1,nlats+1+jj,j-nlats) = temp
      end do
! at same latitude
      do ii=1,nlons
         dlam = real(ii)*dlon
         temp = 6370.0*acos(sin(phi1)*sin(phi1)+cos(phi1)*cos(phi1)*cos(dlam))
         dist(nlons+1-ii,nlats+1,j-nlats) = temp
         dist(nlons+1+ii,nlats+1,j-nlats) = temp
      end do
   end do
!
! weighting for Gaussian smoothing
   w(:,:,:,1) = exp(-1.0*dist*dist/(sigma(1)**2))
   w(:,:,:,2) = exp(-1.0*dist*dist/(sigma(2)**2))
   do j=1,(nlat-2*nlats)
      w(:,:,j,1) = w(:,:,j,1)/sum(w(:,:,j,1))
      w(:,:,j,2) = w(:,:,j,2)/sum(w(:,:,j,2))
   end do
!
! even weighting if within specified distance
   w(:,:,:,3:4) = 0.0
   do j=1,(nlat-2*nlats)
      do jj=1,(nlats+1+nlats)
         do ii=1,(nlons+1+nlons)
            if (dist(ii,jj,j)<=1000) then
               w(ii,jj,j,3) = 1.0
            end if
            if (dist(ii,jj,j)<=500) then
               w(ii,jj,j,4) = 1.0
            end if
         end do
      end do
   end do
   w((nlons+1-10):(nlons+1+10),(nlats+1-10):(nlats+1-6),:,5) = 1.0
   w((nlons+1-10):(nlons+1-6),(nlats+1-10):(nlats+1+10),:,5) = 1.0
   w(nlons+1,nlats+1,:,3) = 0.0
!
   nlow = 0
   varout(:,:,:,:) = 0.0
!
   do k=1,nt
! extend in longitude
      coarse((nlons+1):(nlons+nlon),:) = varin(:,:,k)
      coarse(1:nlons,:) = coarse((nlon+1):(nlon+nlons),:)
      coarse((nlons+nlon+1):(nlon+2*nlons),:) = coarse((nlons+1):(2*nlons),:)
! apply Gaussian smoothing
   do kk=1,2
   do j=(nlats+1),(nlat-nlats)
      do i=1,nlon
         smooth(1+i,j-nlats,kk) = &
         sum(w(:,:,j-nlats,kk)*coarse(i:(nlons+i+nlons),(j-nlats):(j+nlats)))
      end do
   end do
   end do
! extend in longitude
   smooth(1,:,1:2) = smooth(1+nlon,:,1:2)
   smooth(1+nlon+1,:,1:2) = smooth(2,:,1:2)
! remove second smoothed field from first
   smooth(:,:,3) = smooth(:,:,1) - smooth(:,:,2)
!
! extend in longitude
   smoothext(:,:) = 0.0
   smoothext((nlons+1):(nlons+nlon),(nlats+1):(nlat-nlats)) = smooth(2:(1+nlon),:,3)
   smoothext(1:nlons,:) = smoothext((nlon+1):(nlon+nlons),:)
   smoothext((nlons+nlon+1):(nlon+2*nlons),:) = smoothext((nlons+1):(2*nlons),:)
!
   freq(:,:,:) = 0.0
!
! identify extrema
   do j=(nlats+2),(nlat-nlats-1)
      do i=1,nlon
         compare(:,:) = 0.0
         do jj=1,3
            do ii=1,3
               if (smooth(1+i,j-nlats,3)<smooth(1+i-2+ii,j-nlats-2+jj,3)) &
                  compare(ii,jj) = -1.0
               if (smooth(1+i,j-nlats,3)>smooth(1+i-2+ii,j-nlats-2+jj,3)) &
                  compare(ii,jj) =  1.0
            end do
         end do
! local maxima exceeding threshhold and maxima within certain distance
         temp = smooth(1+i,j-nlats,3)  ! same value is smoothext(nlons+i,j)
         tempmax = maxval(smoothext(i:(nlons+i+nlons),(j-nlats):(j+nlats))*w(:,:,j-nlats,3))
         if (sum(compare)==8.0.and.temp>0.00005.and.temp>tempmax) then
            freq(nlons+i,j,1) = freq(i,j,1) + 1.0
            freq(i:(i+2*nlons),(j-nlats):(j+nlats),2:3) = &
            freq(i:(i+2*nlons),(j-nlats):(j+nlats),2:3) + w(:,:,j-nlats,4:5)
            nlow = nlow + 1
            lowinfo(1,nlow) = temp
            lowinfo(2,nlow) = lon(i)
            lowinfo(3,nlow) = lat(j)
            lowinfo(4,nlow) = real(k)
         end if
      end do
   end do
! reflect over prime meridian
   freq((nlons+1):(nlons+nlons),:,:) = freq((nlons+1):(nlons+nlons),:,:) &
                                     + freq((nlons+nlon+1):(nlon+2*nlons),:,:)
   freq((nlon+1):(nlons+nlons),:,:)  = freq((nlon+1):(nlons+nlons),:,:) &
                                     + freq(1:nlons,:,:)
!
   varout(:,:,k,:) = freq((nlons+1):(nlons+nlon),:,:)
!
!   do k=1,nt
   end do
   print *, nlow, sum(lowinfo(2,1:nlow))/nlow
!
! write out to pre-existing file
   status = nf90_open(trim(fileout),NF90_WRITE,ncid)
   status = nf_inq_varid(ncid,'low',varid)
   status = nf_put_var(ncid,varid,varout(:,:,:,1))
   status = nf_inq_varid(ncid,'low500',varid)
   status = nf_put_var(ncid,varid,varout(:,:,:,2))
   status = nf_inq_varid(ncid,'L',varid)
   status = nf_put_var(ncid,varid,varout(:,:,:,3))
   status = nf90_close(ncid)
!
   stop
   end

In [2]:
#import packages
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
#from intake import open_catalog
import cartopy.io.shapereader as shpreader
#import mygrads as mg
import matplotlib as mpl
import cartopy.mpl.ticker as cticker
from cartopy.util import add_cyclic_point
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy.feature import NaturalEarthFeature
from cartopy.feature import COASTLINE
import math
from sklearn.neighbors import NearestNeighbors

In [9]:
filein='/project/xprecip/merra2_ncfiles/slp/anomalies/slp_anoms_traditional_unfiltered_smaller.nc'
DS_slp_anoms=xr.open_dataset(filein)
#DS_slp_anoms
slp_anoms=DS_slp_anoms['SLP']
nlats=len(DS_slp_anoms['lat'])
nlons=len(DS_slp_anoms['lon'])
ntimes=len(DS_slp_anoms['time'])
lat=DS_slp_anoms['lat']
lon=DS_slp_anoms['lon']

In [6]:
#e-folding range for Gaussian smoothing
sigma = (np.arange(150.0,2001.0,1))
sigma
pi = np.arccos(-1.0)
pi

array([ 150.,  151.,  152., ..., 1998., 1999., 2000.])

In [13]:
#compute arc length distance over sub-grid as function of latitude for central grid point

#longitude width in radians
dlon = (360.0/(nlons))*pi/180.0
dist=np.zeros((nlons,nlats,ntimes))
for j in range(nlats+1,(nlats-nlats)):
    phi1 = lat[j]*pi/180.0
    for k in range(1,nlats+1,1):
        phi21 = lat[j-k]*pi/180.0
        phi22 = lat[j+k]*pi/180.0
        for i in range(1,nlons+1,1):
            dlam = i[i]*dlon
            temp = 6370.0*np.arccos(sin(phi1)*sin(phi21)+cos(phi1)*cos(phi21)*cos(dlam))
            temp=dist[nlons+1-i,nlats+1-k,j-nlats]
            temp=dist[nlons+1+i,nlats+1-k,j-nlats]
            temp = 6370.0*np.arccos(sin(phi1)*sin(phi22)+cos(phi1)*cos(phi22)*cos(dlam))
            temp=dist[nlons+1-i,nlats+1+k,j-nlats]
            temp=dist[nlons+1+i,nlats+1+k,j-nlats] 
#at same longitude
        dlam = 0.0
        temp = 6370.0*np.arccos(sin(phi1)*sin(phi21)+cos(phi1)*cos(phi21))
        temp=dist[nlons+1,nlats+1-k,j-nlats]
        temp = 6370.0*np.arccos(sin(phi1)*sin(phi22)+cos(phi1)*cos(phi22))
        temp=dist[nlons+1,nlats+1+k,j-nlats]
#at same latitude
    for i in range(1,nlons+1,1):
        dlam = i[i]*dlon
        temp = 6370.0*np.arccos(sin(phi1)*sin(phi1)+cos(phi1)*cos(phi1)*cos(dlam))
        temp=dist(nlons+1-ii,nlats+1,j-nlats) 
        temp=dist(nlons+1+ii,nlats+1,j-nlats) 
   

In [15]:
#weighting for Gaussian smoothing
w[:,:,:,1] = np.exp(-1.0*dist*dist/(sigma(1)**2))
w[:,:,:,2] = np.exp(-1.0*dist*dist/(sigma(2)**2))
for j in range(1,(nlat-2*nlats)+1):
    w[:,:,j,1] = w[:,:,j,1]/np.sum(w[:,:,j,1])
    w[:,:,j,2] = w[:,:,j,2]/np.sum(w[:,:,j,2])

#even weighting if within specified distance
w[:,:,:,3:4] = 0.0
for j in range(1,(nlat-2*nlats)+1):
    for k in range(1,(nlats+1+nlats)+1):
        for i in range(1,(nlons+1+nlons)+1):
            if (dist[i,k,j]<=1000):
                w[i,k,j,3] = 1.0
            if (dist[i,k,j]<=500):
                w[i,k,j,4] = 1.0
w[(nlons+1-10):(nlons+1+10),(nlats+1-10):(nlats+1-6),:,5] = 1.0
w[(nlons+1-10):(nlons+1-6),(nlats+1-10):(nlats+1+10),:,5] = 1.0
w[nlons+1,nlats+1,:,3] = 0.0
nlow = 0
#varout(:,:,:,:) = 0.0


TypeError: 'numpy.ndarray' object is not callable

In [ ]:
coarse=np.zeros((nlons,nlats))
for k in range(1,ntimes):
#extend in longitude
    coarse[(nlons+1):(nlons+nlon),:] = varin[:,:,k]
    coarse[1:nlons,:] = coarse[(nlon+1):(nlon+nlons),:]
    coarse[(nlons+nlon+1):(nlon+2*nlons),:] = coarse[(nlons+1):(2*nlons),:]
#apply Gaussian smoothing
    for m in range(1,2+1,1):
        for j in range(nlats+1,(nlat-nlats)+1):
            for i in range(1,nlon+1):
                smooth[1+i,j-nlats,m] = np.sum(w[:,:,j-nlats,kk]*coarse[i:(nlons+i+nlons),(j-nlats):(j+nlats)])
#extend in longitude
smooth[1,:,1:2] = smooth[1+nlon,:,1:2]
smooth[1+nlon+1,:,1:2] = smooth[2,:,1:2]
#remove second smoothed field from first
smooth[:,:,3] = smooth[:,:,1] - smooth[:,:,2]

#extend in longitude
smoothext[:,:] = 0.0
smoothext[(nlons+1):(nlons+nlon),(nlats+1):(nlat-nlats)] = smooth[2:(1+nlon),:,3]
smoothext[1:nlons,:] = smoothext[(nlon+1):(nlon+nlons),:]
smoothext[(nlons+nlon+1):(nlon+2*nlons),:] = smoothext[(nlons+1):(2*nlons),:]
freq[:,:,:] = 0.0
